In [94]:
import numpy as np
import pandas as pd
!pip install folium

In [95]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json
from geopy.geocoders import Nominatim
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium

# Import zip code data for Los Angeles County 

In [96]:
import urllib.request
url = 'https://data.lacounty.gov/resource/c3xr-3jw2.json'
page = urllib.request.urlopen(url)
page

In [97]:
from bs4 import BeautifulSoup

In [98]:
soup = BeautifulSoup(page, "lxml")

In [99]:
pretty_soup = soup.prettify()

## REMOVE HTML TAGS AND CLEAN OUT WHITESPACE FROM THE DATA

In [100]:
import re
def remove_html_tags(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

In [101]:
clean_soup = remove_html_tags(pretty_soup)

In [102]:
clean_soup = clean_soup.strip()

In [103]:
print(clean_soup)

[{"zip_code":"90713","postal_city_1":"Lakewood","location":{"latitude":"33.84871142900005","longitude":"-118.11357922799999","human_address":"{\"address\": \"\", \"city\": \"\", \"state\": \"\", \"zip\": \"90713\"}"},":@computed_region_pft9_733t":"2057",":@computed_region_cbw7_skn5":"4",":@computed_region_gj26_y8x3":"17",":@computed_region_x8wy_s94z":"3332"}
,{"zip_code":"91306","postal_city_1":"Winnetka","location":{"latitude":"34.208404020000046","longitude":"-118.57593995299999","human_address":"{\"address\": \"\", \"city\": \"\", \"state\": \"\", \"zip\": \"91306\"}"},":@computed_region_pft9_733t":"282",":@computed_region_cbw7_skn5":"3",":@computed_region_gj26_y8x3":"67",":@computed_region_x8wy_s94z":"4282"}
,{"zip_code":"90002","postal_city_1":"Los Angeles","location":{"latitude":"33.94895070600006","longitude":"-118.24697958699994","human_address":"{\"address\": \"\", \"city\": \"\", \"state\": \"\", \"zip\": \"90002\"}"},":@computed_region_pft9_733t":"1191",":@computed_region_cb

## Turn data into a panda dataframe
Create a list for the zip code, city names, latitude, and longitude then create a panda dataframe from those lists

In [104]:
zip_code = json.loads(clean_soup)

In [105]:
A = [] #list of zip codes
B = [] #list of city names
C = [] #list of latitude
D = [] #list of longitude
for key in zip_code :
    try :
        city = key['not_acceptable_1'] #check if an alternate name for the city exists that might be more useful
        zipcode = key['zip_code']
        A.append(zipcode)
        B.append(city)
        latitude = key['location']['latitude']
        longitude = key['location']['longitude']
        C.append(latitude)
        D.append(longitude)
    except :
        zipcode = key['zip_code']
        city = key['postal_city_1']
        A.append(zipcode)
        B.append(city)
        latitude = key['location']['latitude']
        longitude = key['location']['longitude']
        C.append(latitude)
        D.append(longitude)
    
      

In [106]:
df = pd.DataFrame(A, columns = ['Zip Code'])
df['City'] = B
df['Latitude'] = C
df['Longitude'] = D
df['City plus Zip'] = df['City'] + " " + df['Zip Code']
df

,Zip Code,City,Latitude,Longitude,City plus Zip
0,90713,Lakewood,33.84871142900005,-118.11357922799999,Lakewood 90713
1,91306,Winnetka,34.208404020000046,-118.57593995299999,Winnetka 91306
2,90002,Los Angeles,33.94895070600006,-118.24697958699994,Los Angeles 90002
3,90506,Torrance,33.88535286100006,-118.32659746799999,Torrance 90506
4,90069,West Hollywood,34.08940300900008,-118.37978902499998,West Hollywood 90069
5,91361,Westlake Village,34.15138016700007,-118.83569741699995,Westlake Village 91361
6,90064,Los Angeles,34.03541197100003,-118.43217675599999,Los Angeles 90064
7,91308,West Hills,34.20049176500004,-118.62922710099997,West Hills 91308
8,91702,Azusa,34.12712200200008,-117.90620949499998,Azusa 91702
9,91104,Pasadena,34.16700046200003,-118.12758752599996,Pasadena 91104


In [107]:
#remove all cities that have a zip that starts with 92 or 93. Los Angeles area zips are all 90xxx or 91xxx
for x, y in df.iterrows():
    zipcode = y[0]
    if zipcode[:2] == '92' :
        df.drop(x, inplace= True)
        
    elif zipcode[:2] == '93' :
        df.drop(x, inplace= True)
        
    else: 
        continue
        
df = df.reset_index().drop(columns = ['index'])
df

,Zip Code,City,Latitude,Longitude,City plus Zip
0,90713,Lakewood,33.84871142900005,-118.11357922799999,Lakewood 90713
1,91306,Winnetka,34.208404020000046,-118.57593995299999,Winnetka 91306
2,90002,Los Angeles,33.94895070600006,-118.24697958699994,Los Angeles 90002
3,90506,Torrance,33.88535286100006,-118.32659746799999,Torrance 90506
4,90069,West Hollywood,34.08940300900008,-118.37978902499998,West Hollywood 90069
5,91361,Westlake Village,34.15138016700007,-118.83569741699995,Westlake Village 91361
6,90064,Los Angeles,34.03541197100003,-118.43217675599999,Los Angeles 90064
7,91308,West Hills,34.20049176500004,-118.62922710099997,West Hills 91308
8,91702,Azusa,34.12712200200008,-117.90620949499998,Azusa 91702
9,91104,Pasadena,34.16700046200003,-118.12758752599996,Pasadena 91104


The cities witout a LA County zip code have been dropped from the dataframe.

## Create a map of Los Angeles in Folium

In [108]:
#latitude and longitude coordinates for Los Angeles
lati = 34.0522
long = -118.2437



# create map of Los Angeles using latitude and longitude values
map_los_angeles = folium.Map(location=[lati, long], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df['Latitude'], df['Longitude'], df['City plus Zip']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_los_angeles)  
    
map_los_angeles

Based on the map above, I am going to focus on El Segundo and seven neighboring cities for analysis of opening a coffee shop.

The neighboring cities are :

Playa Del Rey 90293, Play Vista 90045, Inglewood 90301, Inglewood 90304, Hawthorne 90250, Lawndale 90261, Manhattan Beach 90266



# Access Foursquare Data For El Segundo

In [109]:
CLIENT_ID = 'PHABKQVOORZ1WKINXISYFPPTF3LAB4Z5EYIVHK2R5M2H12NR' # your Foursquare ID
CLIENT_SECRET = 'YPNY4GKR3DHWTEN0VMYZCFYTDUIZLNFNBOWWMUWZJYUBDN5Z' # your Foursquare Secret
VERSION = '20180604'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

neighborhood_latitude = df.loc[41, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df.loc[41, 'Longitude'] # neighborhood longitude value
neighborhood_name = df.loc[41, 'City'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))


radius = 1000
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, VERSION, radius, LIMIT)
results = requests.get(url).json()
results

Your credentails:
CLIENT_ID: PHABKQVOORZ1WKINXISYFPPTF3LAB4Z5EYIVHK2R5M2H12NR
CLIENT_SECRET:YPNY4GKR3DHWTEN0VMYZCFYTDUIZLNFNBOWWMUWZJYUBDN5Z
Latitude and longitude values of El Segundo are 33.92015201000004, -118.40580691799994.


{'meta': {'code': 200, 'requestId': '5f25e952c36f8b4e4b04cf88'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'},
    {'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'El Segundo',
  'headerFullLocation': 'El Segundo',
  'headerLocationGranularity': 'city',
  'totalResults': 83,
  'suggestedBounds': {'ne': {'lat': 33.92915201900005,
    'lng': -118.3949813804617},
   'sw': {'lat': 33.911152001000026, 'lng': -118.41663245553818}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '570fd474498eb37edb902e51',
       'name': 'R6 Distillery',
       'location': {'address': '909 E El Segundo Blvd',
        'crossStreet': 'Bungalow Dr',
        'lat': 33.9165283409032,
        'lng': -118.40582188975648,
 

In [110]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# Create a Dataframe of the Venues in El Segundo

In [111]:
#create a dataframe with the nearby venues wih their category
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(10)

,name,categories,lat,lng
0,R6 Distillery,Distillery,33.916528,-118.405822
1,El Segundo Animal Hospital,Pet Store,33.919209,-118.404653
2,Recreation Park,Park,33.920965,-118.412388
3,Ensenada’s Surf N Turf Grill,Taco Place,33.919181,-118.413340
4,La Paz Restaurant,Mexican Restaurant,33.922786,-118.404449
5,The Habit Burger Grill,Burger Joint,33.919988,-118.396484
6,Yellow Brick Road Doggie Playcare & Gym,Pet Store,33.916704,-118.400186
7,El Segundo Brewing Company,Brewery,33.917753,-118.415593
8,Starbucks,Coffee Shop,33.919111,-118.413399
9,Brewport Tap House,Gastropub,33.918093,-118.415759


In [112]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

83 venues were returned by Foursquare.


In [113]:
#the latitude and longitude for El Segundo are 33.92015201000004, -118.40580691799994
lati = 33.920
long = -118.406



# create map of El Segundo area using latitude and longitude values
map_el_segundo = folium.Map(location=[lati, long], zoom_start=14)

# add markers to map
for lat, lng, label in zip(nearby_venues['lat'], nearby_venues['lng'], nearby_venues['categories']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_el_segundo)  
    
map_el_segundo

# Count of Types of Venues in El Segundo

In [114]:
#create a dictionary of the number of times a category is mentioned in El Segundo
categories = nearby_venues['categories']
categories_count = {}
for name in categories :
    selected_row = nearby_venues[nearby_venues["categories"] == name]
    count = len(selected_row)
    categories_count[name] = count
categories_count


{'Distillery': 1,
 'Pet Store': 3,
 'Park': 1,
 'Taco Place': 2,
 'Mexican Restaurant': 4,
 'Burger Joint': 2,
 'Brewery': 1,
 'Coffee Shop': 5,
 'Gastropub': 1,
 'BBQ Joint': 1,
 'IT Services': 1,
 'Italian Restaurant': 1,
 'Sandwich Place': 4,
 'Supermarket': 1,
 'Japanese Restaurant': 3,
 'Hotel': 2,
 'Cuban Restaurant': 1,
 'Sports Bar': 1,
 'Restaurant': 1,
 'American Restaurant': 1,
 'Seafood Restaurant': 1,
 'Mobile Phone Shop': 1,
 'Dance Studio': 2,
 'Diner': 4,
 'Mediterranean Restaurant': 3,
 'Farmers Market': 1,
 'Rental Car Location': 1,
 'Breakfast Spot': 1,
 'Ice Cream Shop': 1,
 'Fast Food Restaurant': 3,
 'Bar': 1,
 'New American Restaurant': 1,
 'Pizza Place': 2,
 'Thai Restaurant': 1,
 'Hotel Bar': 1,
 'Bank': 3,
 'Wine Bar': 1,
 'Salon / Barbershop': 1,
 'Indian Restaurant': 1,
 'Dive Bar': 1,
 'Video Store': 1,
 'Juice Bar': 2,
 'Pharmacy': 2,
 'Latin American Restaurant': 1,
 'Food Truck': 1,
 'French Restaurant': 1,
 'Shipping Store': 1,
 'Vietnamese Restaurant':

In [115]:
#display the categories with the most values
es_venues_sorted = sorted(categories_count.items(), key=lambda x: x[1], reverse=True)
for cat, count in es_venues_sorted :
    print(cat, " ", count)

Coffee Shop   5
Mexican Restaurant   4
Sandwich Place   4
Diner   4
Pet Store   3
Japanese Restaurant   3
Mediterranean Restaurant   3
Fast Food Restaurant   3
Bank   3
Taco Place   2
Burger Joint   2
Hotel   2
Dance Studio   2
Pizza Place   2
Juice Bar   2
Pharmacy   2
Distillery   1
Park   1
Brewery   1
Gastropub   1
BBQ Joint   1
IT Services   1
Italian Restaurant   1
Supermarket   1
Cuban Restaurant   1
Sports Bar   1
Restaurant   1
American Restaurant   1
Seafood Restaurant   1
Mobile Phone Shop   1
Farmers Market   1
Rental Car Location   1
Breakfast Spot   1
Ice Cream Shop   1
Bar   1
New American Restaurant   1
Thai Restaurant   1
Hotel Bar   1
Wine Bar   1
Salon / Barbershop   1
Indian Restaurant   1
Dive Bar   1
Video Store   1
Latin American Restaurant   1
Food Truck   1
French Restaurant   1
Shipping Store   1
Vietnamese Restaurant   1
Convenience Store   1
Athletics & Sports   1
Chinese Restaurant   1
Gym / Fitness Center   1
Gym   1


## El Segundo Discussion:

### Coffee shop is the most common venue in El Segundo. 

### Mexican restaurant, sandwich shop, and diner are in second place.

#### Now lets compare that to the rest of Los Angeles

# Get A List of Venues For All the Neighborhoods of Inglewood 90304

In [116]:
ingl_90304 = df[df['Zip Code'] == '90304']
ingl_90304.reset_index(inplace=True)
index = ingl_90304['index']
index = index[0]

In [117]:

CLIENT_ID = 'PHABKQVOORZ1WKINXISYFPPTF3LAB4Z5EYIVHK2R5M2H12NR' # your Foursquare ID
CLIENT_SECRET = 'YPNY4GKR3DHWTEN0VMYZCFYTDUIZLNFNBOWWMUWZJYUBDN5Z' # your Foursquare Secret
VERSION = '20180604'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

neighborhood_latitude = df.loc[index, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df.loc[index, 'Longitude'] # neighborhood longitude value
neighborhood_name = df.loc[index, 'City'] # neighborhood name
print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))


radius = 1000
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, VERSION, radius, LIMIT)
results = requests.get(url).json()
results

Your credentails:
CLIENT_ID: PHABKQVOORZ1WKINXISYFPPTF3LAB4Z5EYIVHK2R5M2H12NR
CLIENT_SECRET:YPNY4GKR3DHWTEN0VMYZCFYTDUIZLNFNBOWWMUWZJYUBDN5Z
Latitude and longitude values of Inglewood are 33.939383004000035, -118.35812917199996.


{'meta': {'code': 200, 'requestId': '5f25e91e0df21d1c62082be1'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'},
    {'name': '$-$$$$', 'key': 'price'}]},
  'headerLocation': 'Current map view',
  'headerFullLocation': 'Current map view',
  'headerLocationGranularity': 'unknown',
  'totalResults': 47,
  'suggestedBounds': {'ne': {'lat': 33.94838301300005,
    'lng': -118.34730118981476},
   'sw': {'lat': 33.93038299500002, 'lng': -118.36895715418517}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bc4c973b492d13ae691aa60',
       'name': 'La Feria',
       'location': {'address': '10903 S Inglewood Ave',
        'lat': 33.937203814283734,
        'lng': -118.36170317067207,
        'labeledLatLngs': [

# Create a Dataframe of the Venues in Inglewood 90304

In [118]:
#create a dataframe with the nearby venues wih their category

ingl_venues = results['response']['groups'][0]['items']

    
nearby_ingl_venues = json_normalize(ingl_venues) # flatten JSON
nearby_ingl_venues
    # filter columns
filtered_columns = ['venue.name', 'venue.categories','venue.location.city', 'venue.location.postalCode', 'venue.location.lat', 'venue.location.lng']
nearby_ingl_venues =nearby_ingl_venues.loc[:, filtered_columns]

    # filter the category for each row
nearby_ingl_venues['venue.categories'] = nearby_ingl_venues.apply(get_category_type, axis=1)
    # clean columns
nearby_ingl_venues.columns = [col.split(".")[-1] for col in nearby_ingl_venues.columns]

nearby_ingl_venues.head()

,name,categories,city,postalCode,lat,lng
0,La Feria,Mexican Restaurant,Inglewood,90304,33.937204,-118.361703
1,La Fonda,Mexican Restaurant,Inglewood,90304,33.945814,-118.358701
2,Escape Campervans,Rental Car Location,NaN,NaN,33.945290,-118.363352
3,Panda Express,Chinese Restaurant,Inglewood,90304,33.945244,-118.364799
4,Jim Burgers,Burger Joint,Lennox,NaN,33.934384,-118.361154


In [119]:
print('{} venues were returned by Foursquare.'.format(nearby_ingl_venues.shape[0]))

47 venues were returned by Foursquare.


In [120]:
#the latitude and longitude for Inglewood 90304
lati = neighborhood_latitude 
long = neighborhood_longitude

# create map of El Segundo area using latitude and longitude values
map_90304 = folium.Map(location=[lati, long], zoom_start=15)

# add markers to map
for lat, lng, label in zip(nearby_ingl_venues['lat'], nearby_ingl_venues['lng'], nearby_ingl_venues['categories']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_90304)  
    
map_90304

# Count of Types of Venues in Inglewood 90304

In [121]:
#create a dictionary of the number of times a category is mentioned in Inglewood 90304
categories = nearby_ingl_venues['categories']
categories_count = {}
for name in categories :
    selected_row = nearby_ingl_venues[nearby_ingl_venues["categories"] == name]
    count = len(selected_row)
    categories_count[name] = count
categories_count

{'Mexican Restaurant': 6,
 'Rental Car Location': 2,
 'Chinese Restaurant': 3,
 'Burger Joint': 2,
 'Latin American Restaurant': 2,
 'Food Truck': 1,
 'Taco Place': 3,
 'Thai Restaurant': 1,
 'Train Station': 1,
 'Strip Club': 1,
 'Convenience Store': 1,
 'Asian Restaurant': 1,
 'Video Game Store': 1,
 'Pharmacy': 1,
 'Breakfast Spot': 1,
 'Discount Store': 1,
 'Pizza Place': 3,
 'Burrito Place': 1,
 'Sandwich Place': 1,
 'Fast Food Restaurant': 3,
 'Park': 1,
 'Gas Station': 1,
 'Hotel': 3,
 'Airport Terminal': 1,
 'Liquor Store': 4,
 'Hobby Shop': 1}

In [122]:
#display the categories with the most values
ingl_304_venues_sorted = sorted(categories_count.items(), key=lambda x: x[1], reverse=True)
for cat, count in ingl_304_venues_sorted :
    print(cat, " ", count)

Mexican Restaurant   6
Liquor Store   4
Chinese Restaurant   3
Taco Place   3
Pizza Place   3
Fast Food Restaurant   3
Hotel   3
Rental Car Location   2
Burger Joint   2
Latin American Restaurant   2
Food Truck   1
Thai Restaurant   1
Train Station   1
Strip Club   1
Convenience Store   1
Asian Restaurant   1
Video Game Store   1
Pharmacy   1
Breakfast Spot   1
Discount Store   1
Burrito Place   1
Sandwich Place   1
Park   1
Gas Station   1
Airport Terminal   1
Hobby Shop   1


## Inglewood 90304 Discussion:

### Inglewood 90304 does not have any coffee shops listed.

### It is not far from El Segundo and might be a good location if you find an area close enough to the city limits with El Segundo

# Get A List of Venues For All the Neighborhoods of Inglewood 90301

In [123]:
ingl_90301 = df[df['Zip Code'] == '90301']
ingl_90301.reset_index(inplace=True)
index = ingl_90301['index']
index = index[0]

In [124]:
CLIENT_ID = 'PHABKQVOORZ1WKINXISYFPPTF3LAB4Z5EYIVHK2R5M2H12NR' # your Foursquare ID
CLIENT_SECRET = 'YPNY4GKR3DHWTEN0VMYZCFYTDUIZLNFNBOWWMUWZJYUBDN5Z' # your Foursquare Secret
VERSION = '20180604'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

neighborhood_latitude = df.loc[index, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df.loc[index, 'Longitude'] # neighborhood longitude value
neighborhood_name = df.loc[index, 'City'] # neighborhood name
print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))


radius = 1000
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, VERSION, radius, LIMIT)
results = requests.get(url).json()
results

Your credentails:
CLIENT_ID: PHABKQVOORZ1WKINXISYFPPTF3LAB4Z5EYIVHK2R5M2H12NR
CLIENT_SECRET:YPNY4GKR3DHWTEN0VMYZCFYTDUIZLNFNBOWWMUWZJYUBDN5Z
Latitude and longitude values of Inglewood are 33.956363286000055, -118.35484869299995.


{'meta': {'code': 200, 'requestId': '5f25e833fe1f6657f631d6b8'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'},
    {'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Inglewood',
  'headerFullLocation': 'Inglewood',
  'headerLocationGranularity': 'city',
  'totalResults': 42,
  'suggestedBounds': {'ne': {'lat': 33.96536329500007,
    'lng': -118.34401855034614},
   'sw': {'lat': 33.94736327700004, 'lng': -118.36567883565377}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4ba5889af964a520161039e3',
       'name': "Woody's BBQ",
       'location': {'address': '475 S Market St',
        'lat': 33.95743652539731,
        'lng': -118.35235501968566,
        'labeledLatLngs': [{'label': 'display',


# Create a Dataframe of the Venues in Inglewood 90301

In [125]:
#create a dataframe with the nearby venues wih their category
ingl_venues = results['response']['groups'][0]['items']

    
nearby_ingl_venues = json_normalize(ingl_venues) # flatten JSON
nearby_ingl_venues
    # filter columns
filtered_columns = ['venue.name', 'venue.categories','venue.location.city', 'venue.location.postalCode', 'venue.location.lat', 'venue.location.lng']
nearby_ingl_venues =nearby_ingl_venues.loc[:, filtered_columns]

    # filter the category for each row
nearby_ingl_venues['venue.categories'] = nearby_ingl_venues.apply(get_category_type, axis=1)
    # clean columns
nearby_ingl_venues.columns = [col.split(".")[-1] for col in nearby_ingl_venues.columns]

nearby_ingl_venues.head()

,name,categories,city,postalCode,lat,lng
0,Woody's BBQ,BBQ Joint,Inglewood,90301,33.957437,-118.352355
1,Banadir Somali Restaurant,African Restaurant,Inglewood,90301,33.952877,-118.356589
2,Dulan's Soul Food Kitchen,Southern / Soul Food Restaurant,Inglewood,90301,33.961600,-118.352910
3,Stuff I Eat,Vegetarian / Vegan Restaurant,Inglewood,90301,33.963346,-118.351752
4,Inglewood California City Of Champions,Basketball Court,Inglewood,NaN,33.964756,-118.353411


In [126]:
print('{} venues were returned by Foursquare.'.format(nearby_ingl_venues.shape[0]))

42 venues were returned by Foursquare.


In [127]:
#the latitude and longitude for Inglewood 90301
lati = neighborhood_latitude 
long = neighborhood_longitude

# create map of El Segundo area using latitude and longitude values
map_90301 = folium.Map(location=[lati, long], zoom_start=15)

# add markers to map
for lat, lng, label in zip(nearby_ingl_venues['lat'], nearby_ingl_venues['lng'], nearby_ingl_venues['categories']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_90301)  
    
map_90301

# Count of Types of Venues in Inglewood 90301

In [128]:
#create a dictionary of the number of times a category is mentioned in Inglewood 90301
categories = nearby_ingl_venues['categories']
categories_count = {}
for name in categories :
    selected_row = nearby_ingl_venues[nearby_ingl_venues["categories"] == name]
    count = len(selected_row)
    categories_count[name] = count
categories_count

{'BBQ Joint': 1,
 'African Restaurant': 1,
 'Southern / Soul Food Restaurant': 3,
 'Vegetarian / Vegan Restaurant': 1,
 'Basketball Court': 1,
 'Coffee Shop': 2,
 'Pizza Place': 1,
 'Mexican Restaurant': 5,
 'Ice Cream Shop': 1,
 'Taco Place': 1,
 'Caribbean Restaurant': 1,
 'Malay Restaurant': 1,
 'Café': 1,
 'Rental Car Location': 1,
 'Flea Market': 1,
 'Convenience Store': 2,
 'Print Shop': 1,
 'Fast Food Restaurant': 2,
 'Lounge': 1,
 'Mobile Phone Shop': 1,
 'Sandwich Place': 1,
 'Grocery Store': 2,
 'Storage Facility': 1,
 'Burger Joint': 1,
 'Bank': 1,
 'Liquor Store': 1,
 'Donut Shop': 1,
 'Seafood Restaurant': 1,
 'Motel': 1,
 'Bakery': 1,
 'Food Truck': 1,
 'Cajun / Creole Restaurant': 1}

In [129]:
#display the categories with the most values
ingl_301_venues_sorted = sorted(categories_count.items(), key=lambda x: x[1], reverse=True)
for cat, count in ingl_301_venues_sorted :
    print(cat, " ", count)

Mexican Restaurant   5
Southern / Soul Food Restaurant   3
Coffee Shop   2
Convenience Store   2
Fast Food Restaurant   2
Grocery Store   2
BBQ Joint   1
African Restaurant   1
Vegetarian / Vegan Restaurant   1
Basketball Court   1
Pizza Place   1
Ice Cream Shop   1
Taco Place   1
Caribbean Restaurant   1
Malay Restaurant   1
Café   1
Rental Car Location   1
Flea Market   1
Print Shop   1
Lounge   1
Mobile Phone Shop   1
Sandwich Place   1
Storage Facility   1
Burger Joint   1
Bank   1
Liquor Store   1
Donut Shop   1
Seafood Restaurant   1
Motel   1
Bakery   1
Food Truck   1
Cajun / Creole Restaurant   1


## Inglewood 90301 Discussion:

### Mexican Restaurant is the most common venue, followed by Southern/Soul Food. 

### Coffee Shop ranks tied for third most common.

# Get A List of Venues For All the Neighborhoods of Hawthorne 90250

In [130]:
hwth_90250 = df[df['Zip Code'] == '90250']
hwth_90250.reset_index(inplace=True)
index = hwth_90250['index']
index = index[0]

In [131]:
CLIENT_ID = 'PHABKQVOORZ1WKINXISYFPPTF3LAB4Z5EYIVHK2R5M2H12NR' # your Foursquare ID
CLIENT_SECRET = 'YPNY4GKR3DHWTEN0VMYZCFYTDUIZLNFNBOWWMUWZJYUBDN5Z' # your Foursquare Secret
VERSION = '20180604'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

neighborhood_latitude = df.loc[index, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df.loc[index, 'Longitude'] # neighborhood longitude value
neighborhood_name = df.loc[index, 'City'] # neighborhood name
print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))


radius = 1000
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, VERSION, radius, LIMIT)
results = requests.get(url).json()
results

Your credentails:
CLIENT_ID: PHABKQVOORZ1WKINXISYFPPTF3LAB4Z5EYIVHK2R5M2H12NR
CLIENT_SECRET:YPNY4GKR3DHWTEN0VMYZCFYTDUIZLNFNBOWWMUWZJYUBDN5Z
Latitude and longitude values of Hawthorne are 33.914151557000025, -118.34715803399996.


{'meta': {'code': 200, 'requestId': '5f25e974b746232b30fa3c99'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'},
    {'name': '$-$$$$', 'key': 'price'}]},
  'headerLocation': 'Hawthorne',
  'headerFullLocation': 'Hawthorne',
  'headerLocationGranularity': 'city',
  'totalResults': 47,
  'suggestedBounds': {'ne': {'lat': 33.92315156600004,
    'lng': -118.33633325876474},
   'sw': {'lat': 33.90515154800001, 'lng': -118.35798280923518}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bdc8ecbafe8c9b686fc4f85',
       'name': 'Continental Gourmet Market - Hawthorne',
       'location': {'address': '12921 Prairie Ave',
        'crossStreet': 'El Segundo',
        'lat': 33.91493271364134,
        'lng': -11

# Create a Dataframe of the Venues in Hawthorne 90250

In [132]:
#create a dataframe with the nearby venues wih their category
hwth_venues = results['response']['groups'][0]['items']

    
nearby_hwth_venues = json_normalize(hwth_venues) # flatten JSON
nearby_hwth_venues
    # filter columns
filtered_columns = ['venue.name', 'venue.categories','venue.location.city', 'venue.location.postalCode', 'venue.location.lat', 'venue.location.lng']
nearby_hwth_venues =nearby_hwth_venues.loc[:, filtered_columns]

    # filter the category for each row
nearby_hwth_venues['venue.categories'] = nearby_hwth_venues.apply(get_category_type, axis=1)
    # clean columns
nearby_hwth_venues.columns = [col.split(".")[-1] for col in nearby_hwth_venues.columns]

nearby_hwth_venues.head()

,name,categories,city,postalCode,lat,lng
0,Continental Gourmet Market - Hawthorne,Latin American Restaurant,Hawthorne,90250,33.914933,-118.344192
1,Systems Training Center,Martial Arts Dojo,Hawthorne,90250,33.913871,-118.352290
2,Pupuseria El Cerro Verde,Latin American Restaurant,Hawthorne,90250,33.917701,-118.352940
3,Pizza Show,Pizza Place,Hawthorne,90250,33.911181,-118.352213
4,Superior Grocers,Market,Hawthorne,90250,33.917602,-118.351620


In [133]:
print('{} venues were returned by Foursquare.'.format(nearby_hwth_venues.shape[0]))

47 venues were returned by Foursquare.


In [134]:
#the latitude and longitude for Hawthorne 90250
lati = neighborhood_latitude 
long = neighborhood_longitude

# create map of El Segundo area using latitude and longitude values
map_90250 = folium.Map(location=[lati, long], zoom_start=15)

# add markers to map
for lat, lng, label in zip(nearby_hwth_venues['lat'], nearby_hwth_venues['lng'], nearby_hwth_venues['categories']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_90250)  
    
map_90250

# Count of Types of Venues in Hawthorne 90250

In [135]:
#create a dictionary of the number of times a category is mentioned in Hawthorne 90250
categories = nearby_hwth_venues['categories']
categories_count = {}
for name in categories :
    selected_row = nearby_hwth_venues[nearby_hwth_venues["categories"] == name]
    count = len(selected_row)
    categories_count[name] = count
categories_count

{'Latin American Restaurant': 2,
 'Martial Arts Dojo': 2,
 'Pizza Place': 3,
 'Market': 1,
 'Taco Place': 1,
 'Thai Restaurant': 1,
 'Indian Restaurant': 1,
 'Mediterranean Restaurant': 1,
 'Salon / Barbershop': 1,
 'Warehouse Store': 1,
 'Discount Store': 2,
 'Seafood Restaurant': 1,
 'Coffee Shop': 1,
 'Chinese Restaurant': 1,
 'Supplement Shop': 1,
 'Liquor Store': 1,
 'Hotel': 1,
 'Fried Chicken Joint': 1,
 'Mobile Phone Shop': 2,
 'Bakery': 1,
 'Convenience Store': 2,
 'Fast Food Restaurant': 4,
 'Thrift / Vintage Store': 1,
 'Arts & Crafts Store': 1,
 'Big Box Store': 1,
 'Bank': 2,
 'Breakfast Spot': 1,
 'Sandwich Place': 1,
 'Shipping Store': 1,
 'Snack Place': 1,
 'Construction & Landscaping': 1,
 'Smoke Shop': 2,
 'Gym / Fitness Center': 1,
 'Shopping Mall': 1,
 'Mexican Restaurant': 1}

In [136]:
#display the categories with the most values
hwth_90250_venues_sorted = sorted(categories_count.items(), key=lambda x: x[1], reverse=True)
for cat, count in hwth_90250_venues_sorted :
    print(cat, " ", count)

Fast Food Restaurant   4
Pizza Place   3
Latin American Restaurant   2
Martial Arts Dojo   2
Discount Store   2
Mobile Phone Shop   2
Convenience Store   2
Bank   2
Smoke Shop   2
Market   1
Taco Place   1
Thai Restaurant   1
Indian Restaurant   1
Mediterranean Restaurant   1
Salon / Barbershop   1
Warehouse Store   1
Seafood Restaurant   1
Coffee Shop   1
Chinese Restaurant   1
Supplement Shop   1
Liquor Store   1
Hotel   1
Fried Chicken Joint   1
Bakery   1
Thrift / Vintage Store   1
Arts & Crafts Store   1
Big Box Store   1
Breakfast Spot   1
Sandwich Place   1
Shipping Store   1
Snack Place   1
Construction & Landscaping   1
Gym / Fitness Center   1
Shopping Mall   1
Mexican Restaurant   1


## Hawthorne 90250 Discussion:

### Hawthorne 90250 has Fast Food Restaurants as the most common venue, followed by Pizza Place.

### There is only one coffee shop in the Hawthorne area.

# Get A List of Venues For All the Neighborhoods of Lawndale 90261

In [137]:
lwndl_90261 = df[df['Zip Code'] == '90261']
lwndl_90261.reset_index(inplace=True)
index = lwndl_90261['index']
index = index[0]

In [138]:
CLIENT_ID = 'PHABKQVOORZ1WKINXISYFPPTF3LAB4Z5EYIVHK2R5M2H12NR' # your Foursquare ID
CLIENT_SECRET = 'YPNY4GKR3DHWTEN0VMYZCFYTDUIZLNFNBOWWMUWZJYUBDN5Z' # your Foursquare Secret
VERSION = '20180604'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

neighborhood_latitude = df.loc[index, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df.loc[index, 'Longitude'] # neighborhood longitude value
neighborhood_name = df.loc[index, 'City'] # neighborhood name
print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))


radius = 1000
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, VERSION, radius, LIMIT)
results = requests.get(url).json()
results


Your credentails:
CLIENT_ID: PHABKQVOORZ1WKINXISYFPPTF3LAB4Z5EYIVHK2R5M2H12NR
CLIENT_SECRET:YPNY4GKR3DHWTEN0VMYZCFYTDUIZLNFNBOWWMUWZJYUBDN5Z
Latitude and longitude values of Lawndale are 33.89567224900003, -118.37869842799995.


{'meta': {'code': 200, 'requestId': '5f25e9a985d1da39f92442bb'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'},
    {'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Manhattan Village',
  'headerFullLocation': 'Manhattan Village, Manhattan Beach',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 89,
  'suggestedBounds': {'ne': {'lat': 33.90467225800004,
    'lng': -118.36787599897303},
   'sw': {'lat': 33.88667224000002, 'lng': -118.38952085702687}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4a871b9af964a5209e0220e3',
       'name': 'REI',
       'location': {'address': '1800 Rosecrans Ave, Ste E',
        'lat': 33.900527,
        'lng': -118.379719,
        'labeledLatLn

# Create a Dataframe of the Venues in Lawndale 90261

In [139]:
#create a dataframe with the nearby venues wih their category
lwndl_venues = results['response']['groups'][0]['items']

    
nearby_lwndl_venues = json_normalize(lwndl_venues) # flatten JSON
nearby_lwndl_venues
    # filter columns
filtered_columns = ['venue.name', 'venue.categories','venue.location.city', 'venue.location.postalCode', 'venue.location.lat', 'venue.location.lng']
nearby_lwndl_venues =nearby_lwndl_venues.loc[:, filtered_columns]

    # filter the category for each row
nearby_lwndl_venues['venue.categories'] = nearby_lwndl_venues.apply(get_category_type, axis=1)
    # clean columns
nearby_lwndl_venues.columns = [col.split(".")[-1] for col in nearby_lwndl_venues.columns]

nearby_lwndl_venues.head()

,name,categories,city,postalCode,lat,lng
0,REI,Sporting Goods Shop,Manhattan Beach,90266,33.900527,-118.379719
1,Trader Joe's,Grocery Store,Manhattan Beach,90266,33.900699,-118.381144
2,Costco,Warehouse Store,Hawthorne,90250,33.899458,-118.373326
3,Aviation Track,Track,Redondo Beach,90278,33.890203,-118.378018
4,Barnes & Noble,Bookstore,Manhattan Beach,90266,33.900915,-118.380942


In [140]:
print('{} venues were returned by Foursquare.'.format(nearby_lwndl_venues.shape[0]))

89 venues were returned by Foursquare.


In [141]:
#the latitude and longitude for Lawndale 90261
lati = neighborhood_latitude 
long = neighborhood_longitude

# create map of El Segundo area using latitude and longitude values
map_90261 = folium.Map(location=[lati, long], zoom_start=15)

# add markers to map
for lat, lng, label in zip(nearby_lwndl_venues['lat'], nearby_lwndl_venues['lng'], nearby_lwndl_venues['categories']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_90261)  
    
map_90261

# Count of Types of Venues in Lawndale 90261

In [142]:
#create a dictionary of the number of times a category is mentioned in Lawndale 90261
categories = nearby_lwndl_venues['categories']
categories_count = {}
for name in categories :
    selected_row = nearby_lwndl_venues[nearby_lwndl_venues["categories"] == name]
    count = len(selected_row)
    categories_count[name] = count
categories_count


{'Sporting Goods Shop': 1,
 'Grocery Store': 3,
 'Warehouse Store': 2,
 'Track': 1,
 'Bookstore': 2,
 'Optical Shop': 1,
 'Supplement Shop': 2,
 'Mexican Restaurant': 2,
 'Bakery': 1,
 'Coffee Shop': 5,
 'Gym': 3,
 'Health & Beauty Service': 2,
 'Steakhouse': 1,
 'Liquor Store': 1,
 'Clothing Store': 1,
 'Spa': 1,
 'Comic Shop': 1,
 'Italian Restaurant': 2,
 'American Restaurant': 3,
 'Sandwich Place': 2,
 'Ice Cream Shop': 1,
 'Pizza Place': 2,
 'Soccer Field': 1,
 'Mediterranean Restaurant': 1,
 'Bubble Tea Shop': 1,
 'Furniture / Home Store': 2,
 'Burger Joint': 3,
 'Juice Bar': 1,
 'Salad Place': 1,
 'Concert Hall': 1,
 'Burrito Place': 1,
 'Hotel': 4,
 'Video Game Store': 1,
 'Park': 1,
 'Playground': 1,
 'Hardware Store': 1,
 'Hawaiian Restaurant': 1,
 'Construction & Landscaping': 1,
 'Japanese Restaurant': 2,
 'Café': 1,
 'Donut Shop': 1,
 'Arts & Crafts Store': 1,
 'Weight Loss Center': 1,
 'Bank': 2,
 'Seafood Restaurant': 1,
 'Paper / Office Supplies Store': 1,
 'Pharmacy': 

In [143]:
#display the categories with the most values
lwndl_90261_venues_sorted = sorted(categories_count.items(), key=lambda x: x[1], reverse=True)
for cat, count in lwndl_90261_venues_sorted :
    print(cat, " ", count)

Coffee Shop   5
Hotel   4
Grocery Store   3
Gym   3
American Restaurant   3
Burger Joint   3
Fast Food Restaurant   3
Warehouse Store   2
Bookstore   2
Supplement Shop   2
Mexican Restaurant   2
Health & Beauty Service   2
Italian Restaurant   2
Sandwich Place   2
Pizza Place   2
Furniture / Home Store   2
Japanese Restaurant   2
Bank   2
Food Truck   2
Sporting Goods Shop   1
Track   1
Optical Shop   1
Bakery   1
Steakhouse   1
Liquor Store   1
Clothing Store   1
Spa   1
Comic Shop   1
Ice Cream Shop   1
Soccer Field   1
Mediterranean Restaurant   1
Bubble Tea Shop   1
Juice Bar   1
Salad Place   1
Concert Hall   1
Burrito Place   1
Video Game Store   1
Park   1
Playground   1
Hardware Store   1
Hawaiian Restaurant   1
Construction & Landscaping   1
Café   1
Donut Shop   1
Arts & Crafts Store   1
Weight Loss Center   1
Seafood Restaurant   1
Paper / Office Supplies Store   1
Pharmacy   1
Shipping Store   1
Bagel Shop   1
Gas Station   1
Shop & Service   1
Baseball Field   1
Light Rail

## Lawndale 90261 Discussion:

### Lawndale 90261 has Coffee Shop as the most common venue. 

### I would advise not to open a coffee shop here, but perhaps in an adjacent city.

# Get A List of Venues For All the Neighborhoods of Manhattan Beach 90266

In [144]:
mnbch_90266 = df[df['Zip Code'] == '90266']
mnbch_90266.reset_index(inplace=True)
index = mnbch_90266['index']
index = index[0]

In [145]:

CLIENT_ID = 'PHABKQVOORZ1WKINXISYFPPTF3LAB4Z5EYIVHK2R5M2H12NR' # your Foursquare ID
CLIENT_SECRET = 'YPNY4GKR3DHWTEN0VMYZCFYTDUIZLNFNBOWWMUWZJYUBDN5Z' # your Foursquare Secret
VERSION = '20180604'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

neighborhood_latitude = df.loc[index, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df.loc[index, 'Longitude'] # neighborhood longitude value
neighborhood_name = df.loc[index, 'City'] # neighborhood name
print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))


radius = 1000
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, VERSION, radius, LIMIT)
results = requests.get(url).json()
results


Your credentails:
CLIENT_ID: PHABKQVOORZ1WKINXISYFPPTF3LAB4Z5EYIVHK2R5M2H12NR
CLIENT_SECRET:YPNY4GKR3DHWTEN0VMYZCFYTDUIZLNFNBOWWMUWZJYUBDN5Z
Latitude and longitude values of Manhattan Beach are 33.89004112200007, -118.40324770999996.


{'meta': {'code': 200, 'requestId': '5f25e83072cee757fc60dd64'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'},
    {'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Tree Section',
  'headerFullLocation': 'Tree Section, Manhattan Beach',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 92,
  'suggestedBounds': {'ne': {'lat': 33.89904113100008,
    'lng': -118.39242599549796},
   'sw': {'lat': 33.88104111300006, 'lng': -118.41406942450196}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '546517f3498e1cf1aa01dd53',
       'name': "Nick's Manhattan Beach",
       'location': {'address': '451 Manhattan Beach Blvd',
        'crossStreet': 'Morningside Dr.',
        'lat': 33.8861466

# Create a Dataframe of the Venues in Manhattan Beach 90266

In [146]:
#create a dataframe with the nearby venues wih their category
mnbch_venues = results['response']['groups'][0]['items']

    
nearby_mnbch_venues = json_normalize(mnbch_venues) # flatten JSON
nearby_mnbch_venues
    # filter columns
filtered_columns = ['venue.name', 'venue.categories','venue.location.city', 'venue.location.postalCode', 'venue.location.lat', 'venue.location.lng']
nearby_mnbch_venues =nearby_mnbch_venues.loc[:, filtered_columns]

    # filter the category for each row
nearby_mnbch_venues['venue.categories'] = nearby_mnbch_venues.apply(get_category_type, axis=1)
    # clean columns
nearby_mnbch_venues.columns = [col.split(".")[-1] for col in nearby_mnbch_venues.columns]

nearby_mnbch_venues.head()

,name,categories,city,postalCode,lat,lng
0,Nick's Manhattan Beach,American Restaurant,Manhattan Beach,90266,33.886147,-118.408678
1,Lemonade,Restaurant,Manhattan Beach,90266,33.885875,-118.408100
2,Sunlife Organics,Juice Bar,Manhattan Beach,90266,33.885989,-118.408407
3,Shade Hotel,Hotel,Manhattan Beach,90266,33.886744,-118.408872
4,Love & Salt,Italian Restaurant,Manhattan Beach,90266,33.885468,-118.409224


In [147]:
print('{} venues were returned by Foursquare.'.format(nearby_mnbch_venues.shape[0]))

92 venues were returned by Foursquare.


In [148]:
#the latitude and longitude for Manhattan Beach 90266
lati = neighborhood_latitude 
long = neighborhood_longitude

# create map of El Segundo area using latitude and longitude values
map_90266 = folium.Map(location=[lati, long], zoom_start=15)

# add markers to map
for lat, lng, label in zip(nearby_mnbch_venues['lat'], nearby_mnbch_venues['lng'], nearby_mnbch_venues['categories']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_90266)  
    
map_90266

# Count of Types of Venues in Manhattan Beach 90266

In [149]:

#create a dictionary of the number of times a category is mentioned in Manhattan Beach 90266
categories = nearby_mnbch_venues['categories']
categories_count = {}
for name in categories :
    selected_row = nearby_mnbch_venues[nearby_mnbch_venues["categories"] == name]
    count = len(selected_row)
    categories_count[name] = count
categories_count

{'American Restaurant': 4,
 'Restaurant': 1,
 'Juice Bar': 4,
 'Hotel': 3,
 'Italian Restaurant': 3,
 'Gastropub': 3,
 'Farmers Market': 1,
 'Spa': 2,
 'Sushi Restaurant': 2,
 'Coffee Shop': 3,
 'Wine Bar': 1,
 'Optical Shop': 1,
 'New American Restaurant': 1,
 'Seafood Restaurant': 3,
 'Breakfast Spot': 1,
 'Bagel Shop': 1,
 'Pizza Place': 2,
 'Ice Cream Shop': 3,
 'Dessert Shop': 1,
 'Bakery': 2,
 'Pier': 1,
 'Greek Restaurant': 2,
 'Trail': 1,
 'Steakhouse': 1,
 'Sandwich Place': 1,
 'Café': 1,
 'Beer Bar': 1,
 'Pub': 1,
 'Kids Store': 1,
 'Taco Place': 1,
 'Vegetarian / Vegan Restaurant': 1,
 'Asian Restaurant': 1,
 'Big Box Store': 1,
 'Gourmet Shop': 1,
 'Yoga Studio': 1,
 'Japanese Restaurant': 1,
 'Deli / Bodega': 1,
 'Mexican Restaurant': 2,
 'Automotive Shop': 2,
 'Cosmetics Shop': 1,
 'Thai Restaurant': 1,
 'Mobile Phone Shop': 1,
 'College Bookstore': 1,
 'Pilates Studio': 1,
 'Grocery Store': 1,
 'Dive Bar': 2,
 'Video Game Store': 1,
 'Rental Car Location': 1,
 'Sports Ba

In [150]:
#display the categories with the most values
mnbch_90266_venues_sorted = sorted(categories_count.items(), key=lambda x: x[1], reverse=True)
for cat, count in mnbch_90266_venues_sorted :
    print(cat, " ", count)

American Restaurant   4
Juice Bar   4
Hotel   3
Italian Restaurant   3
Gastropub   3
Coffee Shop   3
Seafood Restaurant   3
Ice Cream Shop   3
Bank   3
Spa   2
Sushi Restaurant   2
Pizza Place   2
Bakery   2
Greek Restaurant   2
Mexican Restaurant   2
Automotive Shop   2
Dive Bar   2
Sports Bar   2
Fast Food Restaurant   2
Beach   2
Restaurant   1
Farmers Market   1
Wine Bar   1
Optical Shop   1
New American Restaurant   1
Breakfast Spot   1
Bagel Shop   1
Dessert Shop   1
Pier   1
Trail   1
Steakhouse   1
Sandwich Place   1
Café   1
Beer Bar   1
Pub   1
Kids Store   1
Taco Place   1
Vegetarian / Vegan Restaurant   1
Asian Restaurant   1
Big Box Store   1
Gourmet Shop   1
Yoga Studio   1
Japanese Restaurant   1
Deli / Bodega   1
Cosmetics Shop   1
Thai Restaurant   1
Mobile Phone Shop   1
College Bookstore   1
Pilates Studio   1
Grocery Store   1
Video Game Store   1
Rental Car Location   1
Food Court   1
Bar   1
Shipping Store   1
Pharmacy   1
Indian Restaurant   1
Gym / Fitness Cente

## Manhattan Beach 90266 Discussion: 

### For Manhattan Beach 90266, the most common venue is American Restaurant and Juice Bar. 

### With Coffee Shop being tied for second most common with 6 other venues.

# Get List of Venues for Playa Vista 90045

In [151]:
pv_90045 = df[df['Zip Code'] == '90045']
pv_90045.reset_index(inplace=True)
index = pv_90045['index']
index = index[0]

In [152]:
CLIENT_ID = 'PHABKQVOORZ1WKINXISYFPPTF3LAB4Z5EYIVHK2R5M2H12NR' # your Foursquare ID
CLIENT_SECRET = 'YPNY4GKR3DHWTEN0VMYZCFYTDUIZLNFNBOWWMUWZJYUBDN5Z' # your Foursquare Secret
VERSION = '20180604'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

neighborhood_latitude = df.loc[index, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df.loc[index, 'Longitude'] # neighborhood longitude value
neighborhood_name = df.loc[index, 'City'] # neighborhood name
print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))


radius = 1000
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, VERSION, radius, LIMIT)
results = requests.get(url).json()
results


Your credentails:
CLIENT_ID: PHABKQVOORZ1WKINXISYFPPTF3LAB4Z5EYIVHK2R5M2H12NR
CLIENT_SECRET:YPNY4GKR3DHWTEN0VMYZCFYTDUIZLNFNBOWWMUWZJYUBDN5Z
Latitude and longitude values of Playa Vista are 33.961520807000056, -118.39344972199996.


{'meta': {'code': 200, 'requestId': '5f25eaf2a0f35c7b8882fa97'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'},
    {'name': '$-$$$$', 'key': 'price'}]},
  'headerLocation': 'West Los Angeles',
  'headerFullLocation': 'West Los Angeles, Los Angeles',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 71,
  'suggestedBounds': {'ne': {'lat': 33.97052081600007,
    'lng': -118.38261892277521},
   'sw': {'lat': 33.952520798000045, 'lng': -118.4042805212247}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '49ea58f7f964a52055661fe3',
       'name': 'Truxtons American Bistro',
       'location': {'address': '8611 Truxton Ave',
        'lat': 33.95934544960644,
        'lng': -118.39369745902763,

# Create a Dataframe of the Venues in Playa Vista 90045

In [153]:
#create a dataframe with the nearby venues wih their category
pv_venues = results['response']['groups'][0]['items']

    
nearby_pv_venues = json_normalize(pv_venues) # flatten JSON
nearby_pv_venues 
    # filter columns
filtered_columns = ['venue.name', 'venue.categories','venue.location.city', 'venue.location.postalCode', 'venue.location.lat', 'venue.location.lng']
nearby_pv_venues =nearby_pv_venues .loc[:, filtered_columns]

    # filter the category for each row
nearby_pv_venues['venue.categories'] = nearby_pv_venues .apply(get_category_type, axis=1)
    # clean columns
nearby_pv_venues.columns = [col.split(".")[-1] for col in nearby_pv_venues.columns]

nearby_pv_venues.head()

,name,categories,city,postalCode,lat,lng
0,Truxtons American Bistro,American Restaurant,Los Angeles,90045,33.959345,-118.393697
1,Ayara Thai Cuisine,Thai Restaurant,Los Angeles,90045,33.958740,-118.394530
2,Café Solar,Vegetarian / Vegan Restaurant,Los Angeles,90045,33.959688,-118.394521
3,Coffee Co.,Diner,Los Angeles,90045,33.957505,-118.394699
4,Paco's Tacos,Mexican Restaurant,Los Angeles,90045,33.959651,-118.394127


In [154]:
print('{} venues were returned by Foursquare.'.format(nearby_pv_venues.shape[0]))

71 venues were returned by Foursquare.


In [155]:
#the latitude and longitude for Playa Vista 90045
lati = neighborhood_latitude 
long = neighborhood_longitude

# create map of El Segundo area using latitude and longitude values
map_90045 = folium.Map(location=[lati, long], zoom_start=15)

# add markers to map
for lat, lng, label in zip(nearby_pv_venues['lat'], nearby_pv_venues['lng'], nearby_pv_venues['categories']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_90045)  
    
map_90045

# Count of Types of Venues in Playa Vista 90045

In [156]:
#create a dictionary of the number of times a category is mentioned in Playa Vista 90045
categories = nearby_pv_venues['categories']
categories_count = {}
for name in categories :
    selected_row = nearby_pv_venues[nearby_pv_venues["categories"] == name]
    count = len(selected_row)
    categories_count[name] = count
categories_count

{'American Restaurant': 1,
 'Thai Restaurant': 2,
 'Vegetarian / Vegan Restaurant': 1,
 'Diner': 1,
 'Mexican Restaurant': 2,
 'Soup Place': 1,
 'Grocery Store': 1,
 'Marijuana Dispensary': 1,
 'Sandwich Place': 3,
 'Clothing Store': 1,
 'Japanese Restaurant': 1,
 'Asian Restaurant': 1,
 'Smoothie Shop': 1,
 'Bagel Shop': 1,
 'Record Shop': 1,
 'Bakery': 2,
 'Furniture / Home Store': 2,
 'Fast Food Restaurant': 3,
 'Breakfast Spot': 1,
 'Poke Place': 1,
 'Dessert Shop': 1,
 'Supermarket': 1,
 'Bar': 1,
 'Korean Restaurant': 1,
 'Gym': 1,
 'Park': 1,
 'General Entertainment': 1,
 'Bank': 2,
 'Coffee Shop': 2,
 'Discount Store': 1,
 'Pizza Place': 3,
 'Mobile Phone Shop': 1,
 'Health & Beauty Service': 1,
 'Pet Store': 1,
 'Middle Eastern Restaurant': 1,
 'Frozen Yogurt Shop': 1,
 'Pharmacy': 2,
 'Paper / Office Supplies Store': 2,
 'Shipping Store': 1,
 'Automotive Shop': 1,
 'Video Store': 2,
 'Department Store': 1,
 'Steakhouse': 1,
 'Burrito Place': 1,
 'Hotel': 1,
 'Convenience Stor

In [157]:
#display the categories with the most values
pv_90045_venues_sorted = sorted(categories_count.items(), key=lambda x: x[1], reverse=True)
for cat, count in pv_90045_venues_sorted :
    print(cat, " ", count)

Sandwich Place   3
Fast Food Restaurant   3
Pizza Place   3
Thai Restaurant   2
Mexican Restaurant   2
Bakery   2
Furniture / Home Store   2
Bank   2
Coffee Shop   2
Pharmacy   2
Paper / Office Supplies Store   2
Video Store   2
American Restaurant   1
Vegetarian / Vegan Restaurant   1
Diner   1
Soup Place   1
Grocery Store   1
Marijuana Dispensary   1
Clothing Store   1
Japanese Restaurant   1
Asian Restaurant   1
Smoothie Shop   1
Bagel Shop   1
Record Shop   1
Breakfast Spot   1
Poke Place   1
Dessert Shop   1
Supermarket   1
Bar   1
Korean Restaurant   1
Gym   1
Park   1
General Entertainment   1
Discount Store   1
Mobile Phone Shop   1
Health & Beauty Service   1
Pet Store   1
Middle Eastern Restaurant   1
Frozen Yogurt Shop   1
Shipping Store   1
Automotive Shop   1
Department Store   1
Steakhouse   1
Burrito Place   1
Hotel   1
Convenience Store   1
Shoe Store   1
Bus Station   1
Food Truck   1
Chinese Restaurant   1
Cosmetics Shop   1
Donut Shop   1
Baseball Field   1
Jewish Re

## Playa Vista 90045 Discussion :

### Playa Vista 90045 has Fast Food Restaurant, Sandwich Place, and Pizza Place as the most common venues. 

### Coffee shop is tied for second most common with 8 other categories.

In [158]:
pdr_90293 = df[df['Zip Code'] == '90293']
pdr_90293.reset_index(inplace=True)
index = pdr_90293['index']
index = index[0]

In [159]:
CLIENT_ID = 'PHABKQVOORZ1WKINXISYFPPTF3LAB4Z5EYIVHK2R5M2H12NR' # your Foursquare ID
CLIENT_SECRET = 'YPNY4GKR3DHWTEN0VMYZCFYTDUIZLNFNBOWWMUWZJYUBDN5Z' # your Foursquare Secret
VERSION = '20180604'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

neighborhood_latitude = df.loc[index, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = df.loc[index, 'Longitude'] # neighborhood longitude value
neighborhood_name = df.loc[index, 'City'] # neighborhood name
print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))


radius = 1000
LIMIT = 100
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, neighborhood_latitude, neighborhood_longitude, VERSION, radius, LIMIT)
results = requests.get(url).json()
results

Your credentails:
CLIENT_ID: PHABKQVOORZ1WKINXISYFPPTF3LAB4Z5EYIVHK2R5M2H12NR
CLIENT_SECRET:YPNY4GKR3DHWTEN0VMYZCFYTDUIZLNFNBOWWMUWZJYUBDN5Z
Latitude and longitude values of Playa Del Rey are 33.95744084700004, -118.43916786999995.


{'meta': {'code': 200, 'requestId': '5f25e9437919ac60d9be1fd7'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'},
    {'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Playa del Rey',
  'headerFullLocation': 'Playa del Rey, Los Angeles',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 54,
  'suggestedBounds': {'ne': {'lat': 33.966440856000055,
    'lng': -118.42833759018258},
   'sw': {'lat': 33.94844083800003, 'lng': -118.44999814981732}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b68e42bf964a520d5912be3',
       'name': 'Dales Jr. Market',
       'location': {'address': '8105 W Manchester Ave',
        'crossStreet': 'at Falmouth Ave',
        'lat': 33.95898423757519,
 

# Create a Dataframe of the Venues in Playa Del Rey 90293

In [160]:
#create a dataframe with the nearby venues wih their category
pdr_venues = results['response']['groups'][0]['items']

    
nearby_pdr_venues = json_normalize(pdr_venues) # flatten JSON
nearby_pdr_venues 
    # filter columns
filtered_columns = ['venue.name', 'venue.categories','venue.location.city', 'venue.location.postalCode', 'venue.location.lat', 'venue.location.lng']
nearby_pdr_venues =nearby_pdr_venues .loc[:, filtered_columns]

    # filter the category for each row
nearby_pdr_venues['venue.categories'] = nearby_pdr_venues .apply(get_category_type, axis=1)
    # clean columns
nearby_pdr_venues.columns = [col.split(".")[-1] for col in nearby_pdr_venues.columns]

nearby_pdr_venues.head()

,name,categories,city,postalCode,lat,lng
0,Dales Jr. Market,Liquor Store,Playa del Rey,90293,33.958984,-118.437887
1,Del Rey Deli Co,Deli / Bodega,Playa del Rey,90293,33.956099,-118.442545
2,THE GOOD PIZZA,Pizza Place,Playa del Rey,90293,33.958790,-118.438180
3,Vista del Mar Overlook,Scenic Lookout,NaN,NaN,33.956986,-118.444610
4,Yum Yum Donuts,Donut Shop,Playa del Rey,90293,33.958542,-118.438838


In [161]:
print('{} venues were returned by Foursquare.'.format(nearby_pdr_venues.shape[0]))

54 venues were returned by Foursquare.


In [162]:
#the latitude and longitude for Playa Del Rey 90293
lati = neighborhood_latitude 
long = neighborhood_longitude

# create map of El Segundo area using latitude and longitude values
map_90293 = folium.Map(location=[lati, long], zoom_start=15)

# add markers to map
for lat, lng, label in zip(nearby_pdr_venues['lat'], nearby_pdr_venues['lng'], nearby_pdr_venues['categories']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_90293)  
    
map_90293


# Count of Types of Venues in Playa Del Rey 90293

In [163]:
#create a dictionary of the number of times a category is mentioned in Playa Del Rey 90293
categories = nearby_pdr_venues['categories']
categories_count = {}
for name in categories :
    selected_row = nearby_pdr_venues[nearby_pdr_venues["categories"] == name]
    count = len(selected_row)
    categories_count[name] = count
categories_count


{'Liquor Store': 1,
 'Deli / Bodega': 1,
 'Pizza Place': 3,
 'Scenic Lookout': 2,
 'Donut Shop': 1,
 'Mexican Restaurant': 2,
 'Bar': 1,
 'Restaurant': 1,
 'Indian Restaurant': 1,
 'Burger Joint': 1,
 'Italian Restaurant': 3,
 'Café': 1,
 'Vietnamese Restaurant': 1,
 'Whisky Bar': 1,
 'Beach': 2,
 'Sandwich Place': 1,
 'Juice Bar': 1,
 'Bed & Breakfast': 1,
 'Pilates Studio': 1,
 'Dive Bar': 3,
 'Mediterranean Restaurant': 1,
 'Speakeasy': 2,
 'Convenience Store': 1,
 'Coffee Shop': 2,
 'Massage Studio': 1,
 'Chinese Restaurant': 1,
 'Cosmetics Shop': 1,
 'Automotive Shop': 1,
 'Laundromat': 1,
 'Park': 2,
 'Gym / Fitness Center': 2,
 'Gym': 1,
 'Athletics & Sports': 1,
 'Grocery Store': 1,
 'Flower Shop': 1,
 'Electronics Store': 1,
 'Playground': 1,
 'Accessories Store': 1,
 'Sushi Restaurant': 1,
 'Shopping Mall': 1,
 'Yoga Studio': 1}

In [164]:
#display the categories with the most values
pdr_90293_venues_sorted = sorted(categories_count.items(), key=lambda x: x[1], reverse=True)
for cat, count in pdr_90293_venues_sorted :
    print(cat, " ", count)

Pizza Place   3
Italian Restaurant   3
Dive Bar   3
Scenic Lookout   2
Mexican Restaurant   2
Beach   2
Speakeasy   2
Coffee Shop   2
Park   2
Gym / Fitness Center   2
Liquor Store   1
Deli / Bodega   1
Donut Shop   1
Bar   1
Restaurant   1
Indian Restaurant   1
Burger Joint   1
Café   1
Vietnamese Restaurant   1
Whisky Bar   1
Sandwich Place   1
Juice Bar   1
Bed & Breakfast   1
Pilates Studio   1
Mediterranean Restaurant   1
Convenience Store   1
Massage Studio   1
Chinese Restaurant   1
Cosmetics Shop   1
Automotive Shop   1
Laundromat   1
Gym   1
Athletics & Sports   1
Grocery Store   1
Flower Shop   1
Electronics Store   1
Playground   1
Accessories Store   1
Sushi Restaurant   1
Shopping Mall   1
Yoga Studio   1


## Playa Del Rey 90293 Discussion:

### Playa Del Rey has Pizza Place, Italian Restaurant, and Dive Bar as the most common venues. 

### While Coffee Shop is tied for second with 6 other venues.


# Final Analysis :

Originally, this was to analyze whether opening a coffee shop in El Segundo would be a good idea. Looking at the FourSquare data, El Segundo has many coffee shops and it is the most common venue for that city.

This led me to look at neighboring cities. After analyzing the seven closest cities, I found that Inglewood 90304 has no coffee shops. So starting a coffee shop that is within the Inglewood 90304 area would mean low competition. 

As well, Hawthorne has only one coffee shop, which could be a good indicator of how coffee does in that neighborhood. If the one coffee shop in Hawthorne seems very busy, then it seems like there be enough demand for another coffee shop to open. That may be work.

Looking on the map, the southern region of Inglewood 90304 is adjacent to Hawthorne. Therefor analyzing how business is at the Hawthorne coffee shop should give an good indication as to whether the demand is available in the southern part of Inglewood 90304.



# Recommendation:

## Open a coffee shop in Inglewood 90304, possibly in the southern part of that zip code, or open one in Hawthorne.
